In [1]:
import pandas as pd
import numpy as np
import os
import subprocess # Baixar o arquivo do HDFS para o notebook antes de ler com Pandas

In [10]:
import os
os.getcwd()

'/mnt/notebooks'

In [11]:
os.listdir(".")

['.ipynb_checkpoints',
 'spark-warehouse',
 'teste.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'Untitled3.ipynb',
 'util']

In [12]:
# ============================================================
# 1) Ler o Excel
# ============================================================

hdfs_path = "/user/admin/datasets/cnj/bronze/datamart-2024.xlsx"
local_path = "/mnt/notebooks/datamart-2024.xlsx"

# Baixar o arquivo do HDFS para o local
subprocess.run(["hdfs", "dfs", "-get", "-f", hdfs_path, local_path])

print("Arquivo baixado para:", local_path)


Arquivo baixado para: /mnt/notebooks/datamart-2024.xlsx


In [13]:
os.listdir(".")

['.ipynb_checkpoints',
 'datamart-2024.xlsx',
 'spark-warehouse',
 'teste.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'Untitled3.ipynb',
 'util']

In [33]:
df = pd.read_excel(local_path, dtype=str)
df.head()

,datatamart[procedimento],datatamart[dt_baixa],datatamart[orgao_julgador],datatamart[sigla_grau],datatamart[dt_recebimento],datatamart[processoGeral],datatamart[classe],datatamart[id_classe],datatamart[dt_pendente_liquido]
0,Execução penal não privativa de liberdade,2025-08-18 00:00:00,COCALZINHO DE GOIÁS - Vara Judicial,G1,2024-05-27 00:00:00,7000027-49.2024.8.09.0177,EXECUÇÃO DA PENA,{386},{20240527:20250817}
1,Execução penal privativa de liberdade,2025-08-18 00:00:00,FORMOSA - 1ª Vara Criminal (crimes dolosos con...,G1,2024-12-04 00:00:00,7000457-12.2024.8.09.0044,EXECUÇÃO DA PENA,{386},{20241204:20250817}
2,Execução penal privativa de liberdade,2025-08-18 00:00:00,URUANA - Vara Judicial,G1,2024-04-09 00:00:00,7000014-22.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240409:20250817}
3,Execução penal privativa de liberdade,2025-08-18 00:00:00,URUANA - Vara Judicial,G1,2024-04-19 00:00:00,7000016-89.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240419:20250817}
4,Execução penal privativa de liberdade,2025-08-18 00:00:00,VALPARAÍSO DE GOIAS - 1ª Vara Criminal (crimes...,G1,2024-09-30 00:00:00,7000134-41.2024.8.09.0162,EXECUÇÃO DA PENA,{386},{20240930:20250817}


In [34]:
# ============================================================
# 2) Normalizar nomes das colunas (versão aprimorada)
# ============================================================

def normalizar_colunas(cols):
    cols = (cols
            .str.lower()
            .str.strip()
            # Remover prefixo datatamart[...] 
            .str.replace(r"datatamart\[", "", regex=True)
            .str.replace(r"\]", "", regex=True)
            # Substituir espaços por underline
            .str.replace(" ", "_")
            # Remover caracteres inválidos
            .str.replace(r"[^a-z0-9_]", "", regex=True)
           )
    return cols

# Aplicar ao dataframe
df.columns = normalizar_colunas(df.columns)

In [35]:
df.head()

,procedimento,dt_baixa,orgao_julgador,sigla_grau,dt_recebimento,processogeral,classe,id_classe,dt_pendente_liquido
0,Execução penal não privativa de liberdade,2025-08-18 00:00:00,COCALZINHO DE GOIÁS - Vara Judicial,G1,2024-05-27 00:00:00,7000027-49.2024.8.09.0177,EXECUÇÃO DA PENA,{386},{20240527:20250817}
1,Execução penal privativa de liberdade,2025-08-18 00:00:00,FORMOSA - 1ª Vara Criminal (crimes dolosos con...,G1,2024-12-04 00:00:00,7000457-12.2024.8.09.0044,EXECUÇÃO DA PENA,{386},{20241204:20250817}
2,Execução penal privativa de liberdade,2025-08-18 00:00:00,URUANA - Vara Judicial,G1,2024-04-09 00:00:00,7000014-22.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240409:20250817}
3,Execução penal privativa de liberdade,2025-08-18 00:00:00,URUANA - Vara Judicial,G1,2024-04-19 00:00:00,7000016-89.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240419:20250817}
4,Execução penal privativa de liberdade,2025-08-18 00:00:00,VALPARAÍSO DE GOIAS - 1ª Vara Criminal (crimes...,G1,2024-09-30 00:00:00,7000134-41.2024.8.09.0162,EXECUÇÃO DA PENA,{386},{20240930:20250817}


In [38]:
# ============================================================
# 3) Converter datas para string ISO (YYYY-MM-DD)
#    — converte automaticamente colunas que reconhece como data
# ============================================================
    
# Converter a coluna para datetime
df["dt_recebimento"] = pd.to_datetime(df["dt_recebimento"], errors="coerce")
df["dt_baixa"] = pd.to_datetime(df["dt_baixa"], errors="coerce")

# Converter para string no formato YYYY-MM-DD
df["dt_recebimento"] = df["dt_recebimento"].dt.strftime("%Y-%m-%d")
df["dt_baixa"] = df["dt_baixa"].dt.strftime("%Y-%m-%d")

In [39]:
df.head()

,procedimento,dt_baixa,orgao_julgador,sigla_grau,dt_recebimento,processogeral,classe,id_classe,dt_pendente_liquido
0,Execução penal não privativa de liberdade,2025-08-18,COCALZINHO DE GOIÁS - Vara Judicial,G1,2024-05-27,7000027-49.2024.8.09.0177,EXECUÇÃO DA PENA,{386},{20240527:20250817}
1,Execução penal privativa de liberdade,2025-08-18,FORMOSA - 1ª Vara Criminal (crimes dolosos con...,G1,2024-12-04,7000457-12.2024.8.09.0044,EXECUÇÃO DA PENA,{386},{20241204:20250817}
2,Execução penal privativa de liberdade,2025-08-18,URUANA - Vara Judicial,G1,2024-04-09,7000014-22.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240409:20250817}
3,Execução penal privativa de liberdade,2025-08-18,URUANA - Vara Judicial,G1,2024-04-19,7000016-89.2024.8.09.0154,EXECUÇÃO DA PENA,{386},{20240419:20250817}
4,Execução penal privativa de liberdade,2025-08-18,VALPARAÍSO DE GOIAS - 1ª Vara Criminal (crimes...,G1,2024-09-30,7000134-41.2024.8.09.0162,EXECUÇÃO DA PENA,{386},{20240930:20250817}


In [40]:
# ============================================================
# 4) Salvar CSV local (temporário)
# ============================================================

csv_local = "/mnt/notebooks/processos_bronze.csv"
df.to_csv(csv_local, index=False, encoding="utf-8")

print(f"Arquivo CSV gerado em: {csv_local}")

Arquivo CSV gerado em: /mnt/notebooks/processos_bronze.csv


In [42]:
# ============================================================
# 5) Enviar CSV para o HDFS
# ============================================================

hdfs_destino = "/user/admin/datasets/cnj/bronze/"


# cria o diretório se não existir
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", hdfs_destino])

# envia o arquivo
subprocess.run(["hdfs", "dfs", "-put", "-f", csv_local, hdfs_destino])

print(f"Arquivo enviado ao HDFS: {hdfs_destino}")

Arquivo enviado ao HDFS: /user/admin/datasets/cnj/bronze/
